In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import numpy as np

from ICARUS.core.types import FloatArray
from ICARUS.vehicle.plane import Airplane
from ICARUS.vehicle.surface import WingSurface
from ICARUS.vehicle.utils import SymmetryAxes
from ICARUS.vehicle.plane import Airplane
from ICARUS.vehicle.wing_segment import WingSegment
from ICARUS.computation.solvers.AVL.analyses.polars import avl_angle_run, process_avl_angles_run
from ICARUS.environment.definition import EARTH_ISA
from ICARUS.flight_dynamics.state import State
from ICARUS.vehicle.control_surface import Elevator, Flap

CUDA backend failed to initialize: Unable to load cuPTI. Is it installed? (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
origin= np.array([0.0, 0.0, 0.0], dtype=float)

wing_position= np.array(
    [0.0, 0.0, 0.0],
    dtype=float,
)
wing_orientation=np.array(
    [-1.0, 1.0, 0.0],
    dtype=float,
)

main_wing_flap = Flap(
    span_positions=(0., 0.35),
    hinge_chord_percentages= (0.75, 0.75),
    chord_extension= 1.
)


main_wing = WingSegment(
    name="wing",
    root_airfoil="4415",
    origin=origin + wing_position,
    orientation=wing_orientation,
    symmetries=SymmetryAxes.Y,
    span=2 * 1.130,
    sweep_offset=0,
    root_chord=0.159,
    tip_chord=0.072,
    twist_root=0,
    twist_tip=0,
    N=15,
    M=5,
    mass=0.670,
    controls= [main_wing_flap] 
)

In [4]:


elevator_pos: FloatArray = np.array(
    [0.54, 0.0, 0.0],
    dtype=float,
)
elevator_orientantion: FloatArray = np.array(
    [0.0, 0.0, 0.0],
    dtype=float,
)

elevator = Elevator(
    span_positions= (0., 1.),
    hinge_chord_percentages= (0.5, 0.5)
)

hor_stabilizer = WingSegment(
    name="elevator",
    root_airfoil="NACA0008",
    origin=origin + elevator_pos,
    orientation=elevator_orientantion,
    symmetries=SymmetryAxes.Y,
    span=2 * 0.169,
    sweep_offset=0,
    root_dihedral_angle=0,
    root_chord=0.130,
    tip_chord=0.03,
    N=15,
    M=5,
    mass=0.06,
    controls= [elevator]
)




In [5]:
rudder_position: FloatArray = np.array(
    [0.47, 0.0, 0.01],
    dtype=float,
)
rudder_orientation: FloatArray = np.array(
    [0.0, 0.0, 90.0],
    dtype=float,
)

vertical_stabilizer = WingSegment(
    name="rudder",
    root_airfoil="NACA0008",
    origin=origin + rudder_position,
    orientation=rudder_orientation,
    symmetries=SymmetryAxes.NONE,
    span=0.160,
    sweep_offset=0.1,
    root_dihedral_angle=0,
    root_chord=0.2,
    tip_chord=0.1,
    N=15,
    M=5,
    mass=0.04,
)

In [6]:
lifting_surfaces: list[WingSurface] = [main_wing, hor_stabilizer, vertical_stabilizer]

point_masses = [
    (0.500, np.array([-0.40, 0.0, 0.0], dtype=float), "engine"),  # Engine
    (1.000, np.array([0.090, 0.0, 0.0], dtype=float), "structure"),  # Structure
    # (1.000, np.array([0.090, 0.0, 0.0], dtype=float), "battery"),  # Battery
    (0.900, np.array([0.130, 0.0, 0.0], dtype=float), "payload"),  # Payload
]

In [7]:
planes = []
plane = Airplane(f"test_plane",lifting_surfaces)
plane.add_point_masses(point_masses)
UINF = 20
state = State(
        name="Unstick",
        airplane=plane,
        environment=EARTH_ISA,
        u_freestream= UINF,
)

for d in [0, 10, 20, 30 , 40]:
    solver2D = "Xfoil"
    angles = np.linspace(-10, 10, 11)


    epsilons = {
        "u": 0.01,
        "w": 0.01,
        "q": 0.1,
        "theta": 0.01,
        "v": 0.01,
        "p": 0.1,
        "r": 0.1,
        "phi": 0.01,
    }
    # print(state.control_vars)
    state.set_control(
        {
            "delta_f" : d,
            "delta_e" : 0,  
        }
    )

    state.add_all_pertrubations("Central", epsilons)
    avl_angle_run(
        plane = plane,
        state = state,
        solver2D= "Xfoil",
        angles=angles
    )

Controling wing with {'delta_f': 0}
Controling elevator with {'delta_e': 0}
Controling rudder with {}
Controling wing with {'delta_f': 10}
Controling elevator with {'delta_e': 0}
Controling rudder with {}
Controling wing with {'delta_f': 20}
Controling elevator with {'delta_e': 0}
Controling rudder with {}
	Polar for naca4415_flapped_hinge_0.75_deflection_20.00 not found in database. Trying to recompute
Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                                                           | Description                       |
|-----------|-----------------------------------------------------------------|-----------------------------------|
| airfoil   | Complex Datatype (naca4415_flapped_hinge_0.75_deflection_20.00) | Airfoil Object                    |
| mach      | 0.0                                                             | Mach number    

/home/tryfonas/anaconda3/envs/aero/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


		Total Progres   0%|                              | 0/1 [00:00<?, ?it/s]

Repaneling Airfoil with 120

 Number of input coordinate points: 500
 Counterclockwise ordering
 Max thickness =     0.150085  at x =   0.292
 Max camber    =    -0.039152  at x =   0.760

  LE  x,y  =  -0.00044   0.00181  |   Chord =   0.99817
  TE  x,y  =   0.99408   0.08716  |

 Current airfoil nodes set from buffer airfoil nodes ( 499 )
Adding NACA4415_FLAPPED_HINGE_0.75_DEFLECTION_20.00 to the database
Analysis Completed
	Could not compute polar for naca4415_flapped_hinge_0.75_deflection_20.00
	Polar for naca4415_flapped_hinge_0.75_deflection_20.00 not found in database. Trying to recompute
Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                                                           | Description                       |
|-----------|-----------------------------------------------------------------|-----------------------------------|
| airfoil   | C

/home/tryfonas/anaconda3/envs/aero/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

		Total Progres   0%|                              | 0/1 [00:00<?, ?it/s]

Repaneling Airfoil with 120

 Number of input coordinate points: 500
 Counterclockwise ordering
 Max thickness =     0.150452  at x =   0.292
 Max camber    =    -0.071340  at x =   0.757

  LE  x,y  =  -0.00041   0.00176  |   Chord =   0.98848
  TE  x,y  =   0.97989   0.12864  |

 Current airfoil nodes set from buffer airfoil nodes ( 499 )
Adding NACA4415_FLAPPED_HINGE_0.75_DEFLECTION_30.00 to the database
Analysis Completed
	Could not compute polar for naca4415_flapped_hinge_0.75_deflection_30.00
	Polar for naca4415_flapped_hinge_0.75_deflection_30.00 not found in database. Trying to recompute
Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                                                           | Description                       |
|-----------|-----------------------------------------------------------------|-----------------------------------|
| airfoil   | C

/home/tryfonas/anaconda3/envs/aero/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


		Total Progres   0%|                              | 0/1 [00:00<?, ?it/s]

Repaneling Airfoil with 120

 Number of input coordinate points: 500
 Counterclockwise ordering
 Max thickness =     0.150452  at x =   0.292
 Max camber    =    -0.071340  at x =   0.757

  LE  x,y  =  -0.00041   0.00176  |   Chord =   0.98848
  TE  x,y  =   0.97989   0.12864  |

 Current airfoil nodes set from buffer airfoil nodes ( 499 )
Adding NACA4415_FLAPPED_HINGE_0.75_DEFLECTION_30.00 to the database
Analysis Completed
	Could not compute polar for naca4415_flapped_hinge_0.75_deflection_30.00
Controling wing with {'delta_f': 40}
Controling elevator with {'delta_e': 0}
Controling rudder with {}


In [8]:
from ICARUS.visualization.airplane.db_polars import plot_airplane_polars as plot
# plot([plane.name], solvers = ['AVL'])

(array([[<Axes: title={'center': 'CL vs AoA'}, xlabel='AoA', ylabel='CL'>,
         <Axes: title={'center': 'CD vs AoA'}, xlabel='AoA', ylabel='CD'>],
        [<Axes: title={'center': 'Cm vs AoA'}, xlabel='AoA', ylabel='Cm'>,
         <Axes: title={'center': 'CD vs CL'}, xlabel='CL', ylabel='CD'>],
        [<Axes: >, <Axes: >]], dtype=object),
 <Figure size 1000x1000 with 4 Axes>)